# Bone surface mesh pre-processing
- This notebook cleans and remeshes the raw output of 3D Slicer
- You can skip this notebook if you already have high quality bone surfaces meshes

## imports and user-defined properties

In [1]:
import numpy as np
import meshplot as mp
import os
import igl
from pathlib import Path
import sys

sys.path.append('../')

import src


INPUT SUBJECT INFORMATION:
- model_id = choose between m1 -> m11 to select one of the detaset in the repository.
- i_dim, o_dim: the input and output dimension ( 'mm' = millimeters, 'm' = meters )
- o_format: the format you want the files to be save at ( '.obj' , '.stl' )

In [1]:
# model id
model_id = 'm1'  

# dimensions 
i_dim  = 'mm'     
o_dim  = 'mm'

# file format and suffix
i_format = '.stl'
o_format = '.obj'

DIRECTORIES and PATHS:
- main_dir: locate this directory to where the libhip repository is cloned to.
- i_dir:  the directory of the input bone surface mesh files.
- o_dir:  the directory of the cleaned and remeshed bone model results.

In [3]:
# input and output directories
main_dir = Path('..')

i_dir = main_dir / 'model_repository'/ 'RawSegment'/ model_id
o_dir = main_dir / 'model_generation'/ 'preprocessing_output'/ model_id

config_name  = model_id + '_config'
config_path  = str((main_dir/ 'config'/ config_name).with_suffix('.yml'))
config       = src.Config (config_path)

# Remove all files inside output directory if it exists, otherwise create it
if o_dir.is_dir():
    for file in o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    o_dir.mkdir(exist_ok=False)

In [4]:
# input paths
i_sacrum  = model_id + '_sacrum_'  + i_dim
i_lpelvis = model_id + '_lpelvis_' + i_dim
i_rpelvis = model_id + '_rpelvis_' + i_dim
i_lfemur  = model_id + '_lfemur_'  + i_dim
i_rfemur  = model_id + '_rfemur_'  + i_dim

i_sacrum_path  = str((i_dir/ i_sacrum).with_suffix(i_format))
i_lpelvis_path = str((i_dir/ i_lpelvis).with_suffix(i_format))
i_rpelvis_path = str((i_dir/ i_rpelvis).with_suffix(i_format))
i_lfemur_path  = str((i_dir/ i_lfemur).with_suffix(i_format))
i_rfemur_path  = str((i_dir/ i_rfemur).with_suffix(i_format))

# output paths
o_sacrum  = model_id + '_clean_sacrum_'  + o_dim
o_lpelvis = model_id + '_clean_lpelvis_' + o_dim
o_rpelvis = model_id + '_clean_rpelvis_' + o_dim
o_lfemur  = model_id + '_clean_lfemur_'  + o_dim
o_rfemur  = model_id + '_clean_rfemur_'  + o_dim

o_sacrum_path  = str((o_dir /o_sacrum).with_suffix(o_format))
o_lpelvis_path = str((o_dir /o_lpelvis).with_suffix(o_format))
o_rpelvis_path = str((o_dir /o_rpelvis).with_suffix(o_format))
o_lfemur_path  = str((o_dir /o_lfemur).with_suffix(o_format))
o_rfemur_path  = str((o_dir /o_rfemur).with_suffix(o_format))

REMESHING PARAMS:
- eps: the envelope of size epsilon. Using smaller envelope preserves features better but also takes longer time. 
  The default value of epsilon is b/1000, where b is the length of the diagonal of the bounding box. 
  Please refer to https://github.com/wildmeshing/fTetWild for more information.
- l_* : the ideal edge length. Using smaller ideal edge length gives a denser mesh but also takes longer time. 
  The default ideal edge length is b/20.
  Please refer to https://github.com/wildmeshing/fTetWild for more information.

In [5]:
config_remesh = config.mesh_var

# the envelope of size epsilon
eps      = config_remesh.remesh_epsilon

# edge length for the femur bones
l_leg    = config_remesh.remesh_edge_length_leg

# edge length for the the sacrum and the pelvic bones
l_girdle = config_remesh.remesh_edge_length_girdle


## implementation

### 1- read and clean up bone models
This function first reads an input surface mesh as a set vertices and faces. Next, the duplicated faces and unreferenced vertices are removed from the input bone surface mesh.
- The cleaned models are saved as a set of '_vertices' and '_faces'.
- The number of faces after cleaning is printed out for each bone model.


In [6]:
s_vertices,  s_faces  = src.read_and_clean (i_sacrum_path, i_dim ) 
lp_vertices, lp_faces = src.read_and_clean (i_lpelvis_path, i_dim )
rp_vertices, rp_faces = src.read_and_clean (i_rpelvis_path, i_dim ) 
lf_vertices, lf_faces = src.read_and_clean (i_lfemur_path, i_dim )
rf_vertices, rf_faces = src.read_and_clean (i_rfemur_path, i_dim ) 

number of faces after cleaning 164868
number of faces after cleaning 207040
number of faces after cleaning 206876
number of faces after cleaning 100796
number of faces after cleaning 100472


In [7]:
frame = mp.plot(s_vertices, s_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (lp_vertices, lp_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rp_vertices, rp_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (lf_vertices, lf_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rf_vertices, rf_faces, c = src.bone, shading = src.sh_true)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.021293…

4

### 2- remesh the bone models
We remesh the bone surface mesh to improve the quality and resize the triangles. We use fTetWild to mesh the volume of the surface boundary and re-extract the surface of the volume mesh as the resulting remeshed surface. 
- This meshing method allows vertices of the boundary of the volumetric mesh to move within an epsilon (eps) envelope of the winding number field level-set that defines the surface geometry.
- The number of faces after remeshing is printed out for each bone model.


In [ ]:
rm_s_vertices, rm_s_faces   = src.remesh (s_vertices, s_faces, eps, l_girdle)
rm_lp_vertices, rm_lp_faces = src.remesh (lp_vertices, lp_faces, eps, l_girdle)
rm_rp_vertices, rm_rp_faces = src.remesh (rp_vertices, rp_faces, eps, l_girdle)
rm_lf_vertices, rm_lf_faces = src.remesh (lf_vertices, lf_faces, eps, l_leg)
rm_rf_vertices, rm_rf_faces = src.remesh (rf_vertices, rf_faces, eps, l_leg)

In [ ]:
frame = mp.plot(rm_s_vertices, rm_s_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_lp_vertices, rm_lp_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_rp_vertices, rm_rp_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_lf_vertices, rm_lf_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_rf_vertices, rm_rf_faces, c = src.bone, shading = src.sh_true)

### 3- export results
In this step we save the cleaned bone models suface meshes in '.obj' file format. The output dimension and path is defined by the user.


In [ ]:
src.save_surface (rm_s_vertices, rm_s_faces, o_dim, o_sacrum_path )
src.save_surface (rm_lp_vertices, rm_lp_faces, o_dim, o_lpelvis_path)
src.save_surface (rm_rp_vertices, rm_rp_faces, o_dim, o_rpelvis_path)
src.save_surface (rm_lf_vertices, rm_lf_faces, o_dim, o_lfemur_path )
src.save_surface (rm_rf_vertices, rm_rf_faces, o_dim, o_rfemur_path )